In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
# 경로 설정    
## 공유문서함에 있는 <[SCIC 2021] 삼성카드 공모전> 폴더를 <내 드라이브>에 <드라이브 바로가기 추가>
import os
os.chdir('/content/drive/My Drive/[SCIC 2021] 삼성카드 공모전/2. 샘플 데이터')
os.getcwd()

'/content/drive/My Drive/[SCIC 2021] 삼성카드 공모전/2. 샘플 데이터'

In [3]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 56.6 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
pip install pororo

     |████████████████████████████████| 256 kB 31.9 MB/s 
     |████████████████████████████████| 468 kB 53.9 MB/s 
     |████████████████████████████████| 1.5 MB 61.2 MB/s 
     |████████████████████████████████| 3.1 MB 58.3 MB/s 
     |████████████████████████████████| 5.9 MB 27.8 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 1.7 MB 24.6 MB/s 
     |████████████████████████████████| 270 kB 67.1 MB/s 
     |████████████████████████████████| 65 kB 4.7 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 2.6 MB 41.0 MB/s 
     |████████████████████████████████| 144 kB 54.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 1.2 MB 12.0 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 636 kB 51.1 MB/s 
     |████████████████████████████████| 895 kB 66.6 MB/s 
     |████████████████

In [5]:
# 기본 라이브러리 import
import pandas as pd 
import numpy as np
from datetime import * 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pororo import Pororo

In [6]:
df = pd.read_csv("data_df.csv")
print(df.shape)
df.head()

(1000, 2)


,발화,우선순위
0,상담직원과 빠른연결했슴좋겠어요,5
1,안기다리고. 빨리상담되었음좋겠다,5
2,너무 오래기다리고 상담이 늦어져요,5
3,상담직원과 통화가 매우 힘들다,5
4,상담받으려면오래기다리는게불편함,5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   발화      1000 non-null   object
 1   우선순위    1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [8]:
print(df['발화'].isnull().values.any()) 

False


#### **전처리**

https://wikidocs.net/21693

In [9]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-7ec4j13f
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-7ec4j13f
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=c22dc8a78f4600dbeb5a3e75320e0480eb27cf26f872fc0ab484954d9266bf17
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywzfbvns/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

맞춤법 교정 예시

In [11]:
sent = "무이자할부가 할부가 되는데도 직원이 인지가 안 되어 있는 경우가 많아 많아요요요요"
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

무이자 할부가 할부가 되는데도 직원이 인지가 안 되어 있는 경우가 많아 많아 요요 요요


In [ ]:
sent = "한마디로 짜증 제대로 다시는 삼성렌트 안씁니다  여기도담당아니다 저기도 담당 아니다 정말 어디도 연락이 안됨"
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

새로운 keydata 만들기

In [13]:
from tqdm import tqdm 
import time

In [15]:
Pororo.available_models("summarization")
bullet_summ = Pororo(task="summarization", model="bullet", lang="ko")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.




In [16]:
new_df = pd.DataFrame(columns=['발화', '우선순위'])

for i in tqdm(range(df.shape[0])):
    sent = re.sub('[^a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ1-9]',' ',df['발화'][i])
    spelled_sent = spell_checker.check(sent)
    hanspell_sent = spelled_sent.checked
    if (len(hanspell_sent)>=85):
      keywords = bullet_summ(hanspell_sent)
      hanspell_sent = ''.join(keywords)
    number = list(df['우선순위'])[i]
    new_df = new_df.append(pd.DataFrame([[hanspell_sent, number]], columns=['발화', '우선순위']), ignore_index=True)

  2%|▏         | 23/1000 [00:05<03:25,  4.74it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 1000/1000 [04:00<00:00,  4.16it/s]


In [17]:
new_df

,발화,우선순위
0,상담직원과 빠른연결했슴좋겠어요,5
1,안 기다리고 빨리 상담되었으면 좋겠다,5
2,너무 오래 기다리고 상담이 늦어져요,5
3,상담직원과 통화가 매우 힘들다,5
4,상담받으려면 오래 기다리는 게 불편함,5
...,...,...
995,카드 분실된 지 1분 만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는...,11
996,수시로 6개월 무이자 할부와 링크 혜택이 다른 카드와 구별되는 기능이 편리함,7
997,무이자 할부 자주 많이 주셔서 감사합니다,7
998,무이자 혜택 만족 긴 할부 만족 잘 쓰고 있습니다,7


In [18]:
new_df.to_csv("pororo_df85.csv", index=False)

## **맞춤법**

In [ ]:
length = new_df['발화'].astype(str).apply(len)
plt.hist(length, bins=50, alpha=0.5, color='r', label='word')

In [ ]:
import seaborn as sns
sns.boxplot(length)

In [ ]:
length.describe()